Ноутбук для добавления негативных примеров в конфиг

In [1]:
import os
import pandas as pd
import numpy as np
import json 
import math
import string
import re
from transliterate import translit, get_available_language_codes

from sklearn.model_selection import train_test_split

pd.options.display.max_colwidth=100

In [2]:
RND_SEED = 315

input_dir = 'input'
output_dir = 'output'
input_faq_dir = os.path.join(input_dir, 'faq')
output_faq_dir = os.path.join(output_dir, 'faq')
faq_answers_fname = os.path.join(output_dir, 'faq_answers.json')

negatives_dir = 'fp_fn'
negatives_fname = os.path.join(negatives_dir, 'fp_.xlsx')

In [3]:
for f in os.listdir(output_faq_dir):
    os.remove(os.path.join(output_faq_dir, f))

Чтение исходных данных

In [4]:
negatives = pd.read_excel(negatives_fname)
negatives.head()

,dialog_id,original_text,master_question,mq_id,real_mq_id,distance
180,9268,доброго вечера),Здравствуйте,1011,2002,0.000000e+00
2013,9358,"здравствуйте,как мне привязать карту к другому номеру телефона",Как изменить номер телефона,1013,2002,6.342029e-09
5178,9431,как узнать баланс карты по смс ?,Как узнать баланс карты по смс,1039,2002,8.878841e-09


In [5]:
negatives.head(1).T

,180
dialog_id,9268
original_text,доброго вечера)
master_question,Здравствуйте
mq_id,1011
real_mq_id,2002
distance,0


In [6]:
json_files = os.listdir(input_faq_dir)

In [8]:
cnt_new_negatives = 0
for f in json_files:
    fname_in = os.path.join(input_faq_dir, f)
    fname_out = os.path.join(output_faq_dir, f)
    
    with open(fname_in) as f:
        faq_json = json.load(f, encoding='utf-8')
    
    faq_id = faq_json[0]['id']
    faq_negatives_new = negatives[negatives['mq_id']==faq_id]['original_text'].values.tolist()
    faq_negatives_old = []
    if "samples_negative" in faq_json[0]:
        faq_negatives_old = faq_json[0]["samples_negative"]
    faq_negatives = np.union1d(faq_negatives_old, faq_negatives_new).tolist()
    cnt_new_negatives += len(faq_negatives) - len(faq_negatives_old)
    
    faq_json[0]["samples_negative"] = faq_negatives
#     faq_json[0]["samples_negative"] = []
    with open(fname_out, 'wt', encoding='utf8') as outfile:
        json.dump(faq_json, outfile, indent=4, ensure_ascii=False)

print('New negtives: {}'.format(cnt_new_negatives))

New negtives: 3
